## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import matplotlib

data_path = '../data/Upwind_combine.xlsx'

df_all = pd.read_excel(
    data_path, engine='openpyxl', sheet_name=None)['Sheet1']

In [2]:
name_mapping = {
    # 'Table', ##### Not useful
    # 'OptiDAT nr.', ##### Not useful
    # 'identification', ##### Not useful
    # 'Property Plate', ##### Used to generate Material_Code
    # 'laboratory', ##### Not useful
    'angle': 'Load Angle',
    # 'geometry', ##### Used to generate Material_Code
    # 'material', ##### Not useful
    'average thicknessmiddle': 'Thickness',
    'widthmiddle': 'Width',
    'area[mm2]': 'Area',
    # 'length total[mm]',
    'length gaugeaverage': 'Length(gauge)',
    # 'workpackageI', ##### Not useful
    # 'start date', ##### Not useful
    # 'end date', ##### Not useful
    # 'test type', ##### Identify type
    'R-value': 'Minimum/Maximum Stress',
    'load (max)[kN]': 'Maximum Load',
    'average_emax': 'Average Strain',
    'poissonaverage': 'Average Poisson Ratio',
    'σmax[MPa]': 'Maximum Stress',
    'G[Gpa]': 'Shear Modulus',
    'cycles to failurefatigue': 'Cycles to Failure',
    # 'runout', ##### Not useful
    'loading rate[mm/min]': 'Displacement Rate',
    'loading rate[Hz]': 'Frequency',
    'E_avg[GPa]': 'Modulus (Tensile or Compressive)',
    # 'machine', ##### Not useful
    # 'control mode', ##### Not useful
    # 'special fixture', ##### Not useful
    # 'tabs remarks', ##### Not useful
    'T02 max[ºC]': 'Temperature',
    # 'Environment', ##### Most of them are "Ambient"
    # 'reference document(s)', ##### Not useful
    # 'remarks', ##### Not useful
    # 'invalid', ##### Not useful
    # 'strain gaugesmiddle', ##### Not useful
    # 'grip pressure[MPa]', ##### How to use it?
    'number of layers': 'Number of Layers',
    # 'tab thickness',
    # 'Plate-Fibre Type',
    # 'Plate-Mould', ##### Not useful
    # 'Plate-Date of Arrival', ##### Not useful
    # 'Plate-Glass Density', ##### To calculate Volumn Fraction
    'Plate-Laminate-Width': 'Laminate Width',
    'Plate-Fibre Weight': 'Fibre Weight',
    # 'Plate-Project', ##### Not useful
    'Plate-Laminate-Thickness Adjustment Mould': 'Thickness Adjustment Mould',
    # 'Plate-Injection Date', ##### Not useful
    # 'Plate-Resin System-Mix Ratio', ##### Most of them are the same
    # 'Plate-Material', ##### Not useful
    # 'Plate-Weaver', ##### Not useful
    'Plate-Laminate-Length': 'Laminate Length',
    # 'Plate-Remarks', ##### Not useful
    # 'Plate-Resin Density(liquid)', ##### To calculate Volumn Fraction
    # 'Plate-Fibre Volume Fraction', ##### Incomplete, recalculate
    # 'Plate-Vacuum-Curing', ##### Most of them are the same
    # 'Plate-Layers', ##### Replicated
    # 'Plate-article nr.', ##### Not useful
    'Plate-Prepared Resin Mixture-Resin': 'Resin Mixture Resin Weight',
    # 'Plate-Post-cure', ##### How to use it?
    # 'Plate-Resin System-Resin', ##### Not useful
    'Plate-Vacuum-Injection': 'Vacuum Injection',
    'Plate-Prepared Resin Mixture-Hardener': 'Resin Mixture Hardener Weight',
    'Plate-Lay-up': 'Lay-up',
    # 'Plate-Curing Tabs Glue ', ##### How to use it?
    # 'Plate-Fibre Mass', ##### To calculate Volumn Fraction
    'Plate-Fibre Weight Percentage': 'Fibre Weight Fraction',
    # 'Plate-Delta Cp', ##### What is it?
    # 'Plate-Fibre Weight Fraction', ##### Too many absence
    # 'Plate-Tg', ##### Too many absence
    'Plate-Prepared Resin Mixture-Mix': 'Resin Mixture Weight',
    # 'Plate-Void Content', ##### Too many absence
    'Plate-Prepared Resin Mixture-Ratio': 'Resin Mixture Hardener Fraction',
    # 'Plate-Resin System-Hardener', ##### Not useful
    # 'Plate-Material Density', ##### Too many absence
    # 'Plate-Resin Density(cured)', ##### Data seems not compatible with Resin Density(liquid)
    # 'Plate-Operater' ##### Not useful
    ################### Added features ###################
    'Fibre Volumn Fraction': 'Fibre Volumn Fraction',
    'Maximum Tensile Stress': 'Maximum Tensile Stress',
    'Maximum Compressive Stress': 'Maximum Compressive Stress',
    'Static Maximum Tensile Stress': 'Static Maximum Tensile Stress',
    'Static Maximum Compressive Stress': 'Static Maximum Compressive Stress',
    'Minimum Stress': 'Minimum Stress',
    'Static Elastic Modulus': 'Static Elastic Modulus',
    'Static Compressive Modulus': 'Static Compressive Modulus',
    'Absolute Maximum Stress': 'Absolute Maximum Stress',
    'Absolute Peak-to-peak Stress': 'Absolute Peak-to-peak Stress',
    'Relative Maximum Stress': 'Relative Maximum Stress',
    'Relative Peak-to-peak Stress': 'Relative Peak-to-peak Stress',
}

## Plot initial absence ratio

In [3]:
plot_absence(df_all, name_mapping, '../data/Upwind_absence_ratio_initial.png', fontsize=12)

In [4]:
# Add material code
material_code = [str(df_all.loc[x, 'Property Plate']) + str(df_all.loc[x, 'geometry']) for x in range(len(df_all))]
df_all['Material_Code'] = material_code

In [5]:
# Divide Static Maximum Stress into Tensile and Compressive Stress
df_all['Maximum Tensile Stress'] = np.nan
df_all['Maximum Compressive Stress'] = np.nan
df_all['Elastic Modulus'] = np.nan
df_all['Compressive Modulus'] = np.nan
for idx in range(len(df_all)):
    if df_all.loc[idx,'σmax[MPa]']<0:
        df_all.loc[idx,'Maximum Compressive Stress']=df_all.loc[idx,'σmax[MPa]']
        df_all.loc[idx,'Compressive Modulus']=df_all.loc[idx,'E_avg[GPa]']
    else:
        df_all.loc[idx,'Maximum Tensile Stress']=df_all.loc[idx,'σmax[MPa]']
        df_all.loc[idx,'Elastic Modulus']=df_all.loc[idx,'E_avg[GPa]']

In [6]:
# If R>1, max stress must be negative
where_R1 = []
for idx,x in enumerate(df_all['R-value']):
    if not isinstance(x,str) and x>1:
        where_R1.append(idx)
where_R1 = np.array(where_R1)
df_all.loc[where_R1,'σmax[MPa]'] = -np.abs(df_all.loc[where_R1,'σmax[MPa]'])

In [7]:
# Calculate Minimum Stress using R-value and Maximum Stress
df_all['Minimum Stress'] = df_all['R-value'] * df_all['σmax[MPa]']

## Extract static and fatigue experiments respectively

In [8]:
if 'Plate-Lay-up' in df_all.columns:
    code2seq_dict = {}
    layups = df_all['Plate-Lay-up'].values
    for layer in list(set(layups)):
        code2seq_dict[layer] = code2seq(layer)
    seq = []
    for layer in layups:
        seq.append('/'.join([str(x) for x in code2seq(layer)]))
    df_all['Sequence'] = seq

In [9]:
sequence = [[int(y) if y != '' else np.nan for y in str(x).split('/')] for x in
            df_all['Sequence'].values]
deg_layers = np.zeros((len(sequence), 4),
                      dtype=np.int)  # for 0-deg, pm45-deg, 90-deg, and other directions respectively

for idx, seq in enumerate(sequence):
    deg_layers[idx, 0] = seq.count(0)
    deg_layers[idx, 1] = seq.count(45) + seq.count(-45)
    deg_layers[idx, 2] = seq.count(90)
    deg_layers[idx, 3] = len(seq) - seq.count(np.nan) - np.sum(deg_layers[idx, :3])

deg_layers_name = ['0-deg layers', '45-deg layers', '90-deg layers',
                   'Other-deg layers']
df_all = pd.concat([df_all, pd.DataFrame(data=deg_layers,
                                                  columns=deg_layers_name)], axis=1)

In [10]:
static_indexes = np.union1d(np.where(df_all['test type']=='STT')[0], np.where(df_all['test type']=='STC')[0])
non_static_indexes = np.where(df_all['test type']=='CA')[0]

df_tmp = df_all.copy()

df_static = df_tmp.loc[static_indexes].copy()
df_fatigue = df_tmp.loc[non_static_indexes].copy()

df_static.reset_index(drop=True, inplace=True)
df_fatigue.reset_index(drop=True, inplace=True)

df_fatigue = replace_column_name(df_fatigue, name_mapping)
df_static = replace_column_name(df_static, name_mapping)

In [11]:
static_mat_lay = df_static['Material_Code'].values
static_properties = {}

static_features = ['Maximum Tensile Stress', 'Maximum Compressive Stress', 'Elastic Modulus', 'Compressive Modulus']

for material in list(set(static_mat_lay)):
    where_material = np.where(static_mat_lay == material)[0]
    # print(material, len(where_material))
    material_data = df_static.loc[where_material, static_features].copy()
    material_data.reset_index(drop=True, inplace=True)
    material_df = {}
    for feature in static_features:
        for idx in range(len(material_data[feature])):
            if type(material_data.loc[idx, feature]) == str:
                material_data.loc[idx, feature] = np.nan

        presence_indexes = np.where(material_data[feature])[0]
        mean_value = np.mean(material_data.loc[presence_indexes, feature])
        material_df[feature] = mean_value

    material_df = pd.DataFrame(material_df, index=[0])
    static_properties[material]=material_df

########### modify for another dataset ##############
fatigue_static_features = ['Static '+x if 'Static' not in x else x for x in static_features]
#####################################################
fatigue_mat_lay = df_fatigue['Material_Code'].values

for feature in fatigue_static_features:
    if feature not in list(df_fatigue.columns):
        df_fatigue[feature] = np.nan

for material in list(set(static_mat_lay)):
    where_material = np.where(fatigue_mat_lay == material)[0]
    if len(where_material) > 0:
        static_property = static_properties[material]
        for feature in static_features:
            ########### modify for another dataset ##############
            name = 'Static '+feature if 'Static' not in feature else feature
            feature_absence = np.where(pd.isna(df_fatigue[name]))[0]
            to_assign = np.intersect1d(where_material, feature_absence)
            #####################################################
            df_fatigue.loc[to_assign,name] = static_property[feature].values[0]


In [12]:
df_fatigue['Absolute Maximum Stress'] = np.nan
df_fatigue['Absolute Peak-to-peak Stress'] = np.nan
df_fatigue['Relative Maximum Stress'] = np.nan
df_fatigue['Relative Peak-to-peak Stress'] = np.nan

for idx in range(df_fatigue.values.shape[0]):
    s = np.array([df_fatigue.loc[idx, 'Maximum Stress'],df_fatigue.loc[idx, 'Minimum Stress']])
    which_max_stress = np.where(np.abs(s) == np.max(np.abs(s)))[0]
    if len(which_max_stress) == 0:
        which_max_stress = 1 - int(np.isnan(s[1])) # when nan appears in s
    else:
        which_max_stress = which_max_stress[0]

    relative_to = np.abs(df_fatigue.loc[idx,'Static Maximum Tensile Stress']) if s[which_max_stress] > 0 else np.abs(df_fatigue.loc[idx,'Static Maximum Compressive Stress'])
    if np.isnan(relative_to) and s[0] + s[1] < 1e-5 and s[which_max_stress] > 0:
        relative_to = np.abs(df_fatigue.loc[idx,'Static Maximum Compressive Stress'])

    df_fatigue.loc[idx,'Absolute Maximum Stress'] = s[which_max_stress]
    p2p = np.abs(s[0]-s[1])
    if np.isnan(p2p):
        p2p = np.abs(s[1 - int(np.isnan(s[1]))])
    df_fatigue.loc[idx,'Absolute Peak-to-peak Stress'] = p2p

    if np.abs(s[which_max_stress]/relative_to) <= 1.1: # otherwise static data is not correct
        df_fatigue.loc[idx,'Relative Maximum Stress'] = np.abs(s[which_max_stress]/relative_to)
        df_fatigue.loc[idx,'Relative Peak-to-peak Stress'] = np.abs(p2p/relative_to)
    else:
        df_fatigue.loc[idx,'Static Maximum Tensile Stress'] = np.nan
        df_fatigue.loc[idx,'Static Maximum Compressive Stress'] = np.nan

In [13]:
# Remove runout tests
df_fatigue = df_fatigue.loc[np.where(df_fatigue['runout'] != 'y')[0],:].copy()
df_fatigue.reset_index(drop=True, inplace=True)

In [14]:
df_static['log(Static Maximum Tensile Stress)'] = np.log10(df_static['Maximum Tensile Stress'].values.astype(float))

In [15]:
df_fatigue['log(Cycles to Failure)'] = np.log10(df_fatigue['Cycles to Failure'].values.astype(float))

In [16]:
df_fatigue.to_excel('../data/Upwind_fatigue.xlsx', engine='openpyxl', index=False)
df_static.to_excel('../data/Upwind_static.xlsx', engine='openpyxl', index=False)

In [17]:
df_static = replace_column_name(df_static, name_mapping)
averaging(df_fatigue, measure_features=['Maximum Stress', 'Minimum Stress', 'Frequency']).to_excel('../data/Upwind_avg_fatigue.xlsx', engine='openpyxl', index=False)
averaging(df_static, measure_features=['Displacement Rate']).to_excel('../data/Upwind_avg_static.xlsx', engine='openpyxl', index=False)

  0%|          | 0/70 [00:00<?, ?it/s]

/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: Ru

  0%|          | 0/62 [00:00<?, ?it/s]

/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/xlluo/anaconda3/envs/ML-fracture/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [18]:
plot_absence(df_fatigue, name_mapping, '../data/Upwind_absence_ratio.png', fontsize=5)

In [19]:
plot_absence(df_static, name_mapping, '../data/Upwind_static_absence_ratio.png', fontsize=5)